# Client

Demo of a client interacting with a remote agent that can use history.

See relevant documentation about agents:

* Creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
* Streaming with agents: https://python.langchain.com/docs/modules/agents/how_to/streaming#custom-streaming-with-events
* General streaming documentation: https://python.langchain.com/docs/expression_language/streaming

You can interact with this via API directly

In [2]:
!pip install sse_starlette

In [10]:
import requests

inputs = {
    "input": {
        "input": "What are the categories in the database?",
        "chat_history": [],
    }
}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': {'output': 'The word "categories" has 10 letters. Let me check the database for the categories.\nI have found the categories in the database.'},
 'metadata': {'run_id': 'f216494e-7474-4a9e-855d-cfa0c27b434c',
  'feedback_tokens': []}}

In [11]:
print(response.json()["output"]["output"])

The word "categories" has 10 letters. Let me check the database for the categories.
I have found the categories in the database.


You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [12]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [13]:
from langchain_core.messages import HumanMessage, AIMessage

In [14]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break
    ai = await remote_runnable.ainvoke({"input": human, "chat_history": chat_history})
    print(f"AI: {ai['output']}")
    chat_history.extend([HumanMessage(content=human), AIMessage(content=ai["output"])])

AI: Hi there! How can I assist you today?
AI: The word "categories" has 10 letters. Is there anything else you would like to know?
AI: ¡Hola! ¿En qué puedo ayudarte hoy?
AI: Bye bye human


In [10]:
chat_history

[HumanMessage(content='what are the profiles with tag plomero?'),
 AIMessage(content='I\'m sorry, but I couldn\'t find any profiles with the tag "plomero." If you have any other requests or need assistance with something else, feel free to let me know!'),
 HumanMessage(content='profiles with tag: plomero'),
 AIMessage(content='I apologize for the inconvenience. It seems there was an issue with retrieving profiles with the tag "plomero." If you have any other specific tags or requests, please let me know so I can assist you further.'),
 HumanMessage(content='plomero'),
 AIMessage(content='I found profiles with the tag "plomero" in the following categories:\n- Cerrajería\n- 8_56_Hierro\n- Reparación\n- Electricidad\n- Plomería\n\nIf you would like more information about any specific profile or category, feel free to ask!')]

## Stream

Please note that streaming alternates between actions and observations. It does not stream individual tokens!

To stream individual tokens, we need to use the astream events endpoint (see below).

In [19]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream(
        {"input": human, "chat_history": chat_history}
    ):
        # Agent Action
        if "actions" in chunk:
            for action in chunk["actions"]:
                print(
                    f"Calling Tool ```{action['tool']}``` with input ```{action['tool_input']}```"
                )
        # Observation
        elif "steps" in chunk:
            for step in chunk["steps"]:
                print(f"Got result: ```{step['observation']}```")
        # Final result
        elif "output" in chunk:
            print(chunk["output"])
            ai = AIMessage(content=chunk["output"])
        else:
            raise ValueError
        print("------")
    chat_history.extend([HumanMessage(content=human), ai])

AI: 
¡Hola! ¿En qué puedo ayudarte hoy?
------
AI: 
Calling Tool ```service_categories``` with input ```{}```
------
Got result: ```['misc', 'cerrajeria', 'piano', 'cabello', 'bateria', 'french_food', 'body_treatments', '8_56_hierro', 'home', 'moto', 'filmmaking', 'auto', 'music', 'video', 'aseo_completo_de_casas', 'cubana', 'uber', 'tutorias', 'muebles', 'gatos', 'transporte_de_alimentos', 'smartphone', 'carrozas', 'plomero', 'taxi', 'delivery', 'equipos_de_sonido', 'hamburguesas', '5', 'education', 'tv', 'motoboy', 'piano', 'musica', 'albanil', 'depilacion', 'llaves_perdidas', 'mototaxi', 'guitar', 'restaurant', 'moda', 'food', 'manos_y_pies', 'indriver', 'cerrajeria_automotriz', 'foodtruck', 'marketing', 'electeicidade_generale', 'repair', 'jardin', 'travel_agency', 'transport', 'piano', 'alba', 'musica', 'computadoras', 'pies', 'bateria', 'nevera', 'musico', 'speak_english', 'doctor', 'piano', 'psicopedagoga', 'electricidade', 'bicicletas', 'aire_aconditionado', 'estufas', 'musica'

In [20]:
chat_history

[HumanMessage(content='Hola'),
 AIMessage(content='¡Hola! ¿En qué puedo ayudarte hoy?'),
 HumanMessage(content='Necesito alguien que me lave el auto'),
 AIMessage(content='Puedo ayudarte a encontrar un servicio de lavado de automóviles. ¿Te gustaría que busque un proveedor de este servicio para ti?')]

## Stream Events

In [18]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print("AI: Bye bye human")
        break
    ai = None
    print("AI: ")
    async for event in remote_runnable.astream_events(
        {"input": human, "chat_history": chat_history},
        version="v1",
    ):
        kind = event["event"]
        if kind == "on_chain_start":
            if (
                event["name"] == "agent"
            ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
                print(
                    f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
                )
        elif kind == "on_chain_end":
            if (
                event["name"] == "agent"
            ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
                print()
                print("--")
                print(
                    f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
                )
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # Empty content in the context of OpenAI means
                # that the model is asking for a tool to be invoked.
                # So we only print non-empty content
                print(content, end="|")
        elif kind == "on_tool_start":
            print("--")
            print(
                f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
            )
        elif kind == "on_tool_end":
            print(f"Done tool: {event['name']}")
            print(f"Tool output was: {event['data'].get('output')}")
            print("--")

AI: 
Starting agent: agent with input: {'input': 'Hola', 'chat_history': []}
¡|Hola|!| ¿|En| qué| puedo| ayud|arte| hoy|?|
--
Done agent: agent with output: ¡Hola! ¿En qué puedo ayudarte hoy?
AI: 
Starting agent: agent with input: {'input': 'Hay mecanicos de autos como provehedores de servicios?', 'chat_history': []}
--
Starting tool: service_categories with inputs: {}
Done tool: service_categories
Tool output was: ['misc', 'cerrajeria', 'piano', 'cabello', 'bateria', 'french_food', 'body_treatments', '8_56_hierro', 'home', 'moto', 'filmmaking', 'auto', 'music', 'video', 'aseo_completo_de_casas', 'cubana', 'uber', 'tutorias', 'muebles', 'gatos', 'transporte_de_alimentos', 'smartphone', 'carrozas', 'plomero', 'taxi', 'delivery', 'equipos_de_sonido', 'hamburguesas', '5', 'education', 'tv', 'motoboy', 'piano', 'musica', 'albanil', 'depilacion', 'llaves_perdidas', 'mototaxi', 'guitar', 'restaurant', 'moda', 'food', 'manos_y_pies', 'indriver', 'cerrajeria_automotriz', 'foodtruck', 'marketin